In [1]:
import requests

In [21]:
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

ny = geocode("New York")
ny

{'type': 'Point', 'coordinates': [-73.9586, 40.68908]}

In [22]:
# https://docs.mongodb.com/manual/reference/operator/query/near/
def withGeoQuery(location,maxDistance=10000,minDistance=0,field="location"):
    return {
       field: {
         "$near": {
           "$geometry": location if type(location)==dict else geocode(location),
           "$maxDistance": maxDistance,
           "$minDistance": minDistance
         }
       }
    }



In [23]:
from pprint import pprint 

query = withGeoQuery(ny)
pprint(query)

{'location': {'$near': {'$geometry': {'coordinates': [-73.9586, 40.68908],
                                      'type': 'Point'},
                        '$maxDistance': 10000,
                        '$minDistance': 0}}}


In [5]:
withGeoQuery("Madrid")

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': ['-3.67932', '40.42953']},
   '$maxDistance': 10000,
   '$minDistance': 0}}}

In [6]:
query

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': ['-73.95860', '40.68908']},
   '$maxDistance': 10000,
   '$minDistance': 0}}}

In [7]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [8]:
companies = list(db["crunchbase"].find({},{"name":1,"offices":1,"category_code":1}))


In [34]:
import pandas as pd
df = pd.DataFrame(companies)
display(df)


,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8c,Zoho,software,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,"[{'description': '', 'address1': '710 - 2nd Av..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"[{'description': '', 'address1': 'Suite 200', ..."
4,52cdef7c4bab8bd675297d90,Postini,web,"[{'description': None, 'address1': '959 Skyway..."
...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f6,Adhunk,advertising,"[{'description': 'Indian Office', 'address1': ..."
18797,52cdef7f4bab8bd67529c6f7,AfterLogic,software,"[{'description': 'Livingston', 'address1': 'P...."
18798,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,"[{'description': '', 'address1': '', 'address2..."
18799,52cdef7f4bab8bd67529c6fa,EnteGreat,enterprise,"[{'description': '', 'address1': '1900 Interna..."


In [35]:
df = df.explode('offices')
dfOfficeData = df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
cleanData = pd.concat([df,dfOfficeData], axis=1)
display(cleanData)
display(df)
display(dfOfficeData)

,_id,name,category_code,offices,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"{'description': 'Headquarters', 'address1': '4...",Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,52cdef7c4bab8bd675297d8c,Zoho,software,"{'description': 'Headquarters', 'address1': '4...",Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave...",,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette...",,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"{'description': '', 'address1': 'Suite 200', '...",,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f6,Adhunk,advertising,"{'description': 'Indian Office', 'address1': '...",Indian Office,Near Tube Well(south),Patel Nagar,334003,Bikaner,None,IND,NaN,NaN
18797,52cdef7f4bab8bd67529c6f7,AfterLogic,software,"{'description': 'Livingston', 'address1': 'P.O...",Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554
18798,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,"{'description': '', 'address1': '', 'address2'...",,,,,Birmingham,AL,USA,33.518885,-86.816068
18799,52cdef7f4bab8bd67529c6fa,EnteGreat,enterprise,"{'description': '', 'address1': '1900 Internat...",,1900 International Park Drive,Suite 200,35243-4204,Birmingham,AL,USA,33.407786,-86.761302


,_id,name,category_code,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,"{'description': 'Headquarters', 'address1': '4..."
1,52cdef7c4bab8bd675297d8c,Zoho,software,"{'description': 'Headquarters', 'address1': '4..."
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '710 - 2nd Ave..."
2,52cdef7c4bab8bd675297d8a,Wetpaint,web,"{'description': '', 'address1': '270 Lafayette..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,"{'description': '', 'address1': 'Suite 200', '..."
...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f6,Adhunk,advertising,"{'description': 'Indian Office', 'address1': '..."
18797,52cdef7f4bab8bd67529c6f7,AfterLogic,software,"{'description': 'Livingston', 'address1': 'P.O..."
18798,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,"{'description': '', 'address1': '', 'address2'..."
18799,52cdef7f4bab8bd67529c6fa,EnteGreat,enterprise,"{'description': '', 'address1': '1900 Internat..."


,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
2,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
3,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
...,...,...,...,...,...,...,...,...,...
18796,Indian Office,Near Tube Well(south),Patel Nagar,334003,Bikaner,None,IND,NaN,NaN
18797,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554
18798,,,,,Birmingham,AL,USA,33.518885,-86.816068
18799,,1900 International Park Drive,Suite 200,35243-4204,Birmingham,AL,USA,33.407786,-86.761302


In [12]:
cleanData = cleanData.drop(columns=["_id","offices"])

In [13]:
import math
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
cleanData[["latitude","longitude","location"]]

,latitude,longitude,location
0,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,37.692934,-121.904945,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
2,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,NaN,NaN,None
...,...,...,...
18796,NaN,NaN,None
18797,40.793024,-74.323554,"{'type': 'Point', 'coordinates': [-74.3235539,..."
18798,33.518885,-86.816068,"{'type': 'Point', 'coordinates': [-86.816068, ..."
18799,33.407786,-86.761302,"{'type': 'Point', 'coordinates': [-86.7613019,..."


In [14]:
cleanData = cleanData.drop(columns=["latitude","longitude"])

In [15]:
cleanData.head()

,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location
0,AdventNet,enterprise,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
1,Zoho,software,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,"{'type': 'Point', 'coordinates': [-121.904945,..."
2,Wetpaint,web,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,"{'type': 'Point', 'coordinates': [-122.333253,..."
2,Wetpaint,web,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.9964312,..."
3,Omnidrive,network_hosting,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,None


In [16]:
# When exporting data for mongoimport, REMEMBER! -> orient=records
cleanData.to_json("data/cleaned_offices.json", orient="records")

# Next step: 
# mongoimport --db companies --collection cleancompanies --jsonArray --drop cleaned_offices.json

## Create geospartial index
# Indexes > Create Index > (write any indexname) > Select fieldname:"location" and 2dsphere

In [25]:
# GEOQUERY: Companies 10Km near Times Square
comp = list(db["cleancompanies"].find(query))
print(len(comp))

646


In [30]:
pd.DataFrame(comp).drop(columns=["_id"]).to_json("data/filtered.json", orient="records")